<a href="https://colab.research.google.com/github/Idan-Alter/OU-22961-Deep-Learning/blob/main/22961_6_2_embedding_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 77 kB 4.6 MB/s 
     |████████████████████████████████| 212 kB 66.2 MB/s 
     |████████████████████████████████| 1.1 MB 10.5 MB/s 
     |████████████████████████████████| 136 kB 47.4 MB/s 
     |████████████████████████████████| 127 kB 46.3 MB/s 
     |████████████████████████████████| 271 kB 44.2 MB/s 
     |████████████████████████████████| 144 kB 45.4 MB/s 
     |████████████████████████████████| 94 kB 410 kB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import torch
from torch import nn
import datasets as ds
from pprint import pprint
from tqdm import tqdm

dataset = ds.load_dataset("glue", "sst2")

sentence_list = dataset["train"]["sentence"]
labels_list   = dataset["train"]["label"]
tokenize      = lambda x: x.split()
tokenized     = list(map(tokenize, sentence_list))

from torchtext.vocab import build_vocab_from_iterator
vocab = build_vocab_from_iterator(tokenized, specials=["<UNK>"], min_freq=5)
vocab.set_default_index(1)

func = lambda x: torch.tensor(vocab(x))
integer_tokens = list(map(func, tokenized))
label_tensors  = list(map(torch.tensor, labels_list))
print(*sentence_list[1:3], sep="\n")
print(*integer_tokens[1:3], sep="\n")
print(*label_tensors[1:3], sep="\n")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

contains no wit , only labored gags 
that loves its characters and communicates something rather beautiful about human nature 
tensor([2924,   61,  330,    2,   89, 1993,  549])
tensor([  10, 1792,   17,   54,    4, 6088,   96,  186,  265,   34,  178,  627])
tensor(0)
tensor(1)


In [ ]:
test_split = len(integer_tokens) * 8//10
train_tokens = integer_tokens[:test_split]
train_labels = label_tensors[:test_split]
test_tokens  = integer_tokens[test_split:]
test_labels  = label_tensors[test_split:]

In [ ]:
class ClassificationHead(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 2)
        self.logsoftmax = nn.LogSoftmax(dim=0)

    def forward(self, feature_extractor_output):
        class_scores= self.linear(feature_extractor_output)
        logprobs    = self.logsoftmax(class_scores)
        return logprobs

In [ ]:
class FeatureExtractor_1(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(len(vocab),embed_dim)

    def forward(self, sentence_tokens):
        embedded    = self.embedding(sentence_tokens)
        return embedded

In [ ]:
example_sentence=sentence_list[1]

In [ ]:
print(example_sentence)

preprocess= lambda x: torch.tensor(vocab(x.split()))
tokens = preprocess(example_sentence)
print(tokens)

extractor = FeatureExtractor_1(2)
features = extractor(tokens)
print(features, features.size(), sep="\n")


contains no wit , only labored gags 
tensor([2924,   61,  330,    2,   89, 1993,  549])
tensor([[ 1.6337,  0.7351],
        [-0.9232,  0.3840],
        [ 0.5395,  0.2960],
        [-0.5276,  1.6171],
        [ 0.2901,  2.6462],
        [ 1.5971, -0.2344],
        [-0.3403,  0.2870]], grad_fn=<EmbeddingBackward0>)
torch.Size([7, 2])


In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(len(vocab), embed_dim)

    def forward(self, sentence_tokens):
        embedded    = self.embedding(sentence_tokens)
        feature_extractor_output = embedded.sum(dim=0)    #
        return feature_extractor_output

In [ ]:
extractor = FeatureExtractor(2)
features  = extractor(tokens)
print(features, features.size(), sep="\n")

tensor([ 0.9963, -1.0662], grad_fn=<SumBackward1>)
torch.Size([2])


In [ ]:
class EmbedSumClassify(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.extractor  = FeatureExtractor(embed_dim)
        self.classifier = ClassificationHead(embed_dim)

    def forward(self, sentence_tokens):
        extracted_features = self.extractor(sentence_tokens)
        logprobs    = self.classifier(extracted_features)
        return logprobs

In [ ]:
model = EmbedSumClassify(2)
print(model(tokens))

tensor([-0.0870, -2.4856], grad_fn=<LogSoftmaxBackward0>)


In [ ]:
def iterate_one_sentence(tokens, label, train_flag):
  tokens = tokens
  if train_flag:
    model.train()  
    optimizer.zero_grad()
    y_model = model(tokens)
    loss    = -y_model[label] #CE loss
    loss.backward()
    optimizer.step()
  else:
    model.eval()
    y_model = model(tokens)
    model.train()
  with torch.no_grad():
    predicted_labels = y_model.argmax(dim=0)
    success = (predicted_labels == label)
  return success

def train_one_epoch():
  correct_predictions = torch.tensor([0.])
  for tokens, label in tqdm(zip(train_tokens, train_labels), total=len(train_tokens)):
    correct_predictions += iterate_one_sentence(tokens,label, train_flag=True)
  acc = correct_predictions / len(train_tokens)
  print("\n", acc)
  return acc

def test_model():
  test_correct_predictions = torch.tensor([0.])
  for tokens, label in tqdm(zip(test_tokens, test_labels), total=len(test_tokens)):
    test_correct_predictions += iterate_one_sentence(tokens, label, train_flag=False)
  test_acc = test_correct_predictions / len(test_tokens)
  return test_acc

In [ ]:
model = EmbedSumClassify(5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

acc=train_one_epoch()
test_acc=test_model()

100%|██████████| 53879/53879 [01:48<00:00, 496.76it/s]



 tensor([0.7621])


100%|██████████| 13470/13470 [00:02<00:00, 5904.77it/s]


In [ ]:
#check on random labels
test_correct_predictions = torch.tensor([0.])
random_labels = torch.rand(len(test_tokens))<0.5
for tokens, label in tqdm(zip(test_tokens, random_labels), total=len(test_tokens)):
  test_correct_predictions += iterate_one_sentence(tokens, label, train_flag=False)
rand_acc = test_correct_predictions / len(test_tokens)

100%|██████████| 13470/13470 [00:02<00:00, 5728.82it/s]


In [ ]:
print(acc, test_acc, rand_acc, sep="\n")

tensor([0.7621])
tensor([0.8223])
tensor([0.5004])


In [ ]:
preprocess = lambda x: torch.tensor(vocab(x.split()))
example_sentences=["very good , not bad",
                   "very bad , not good"]
with torch.no_grad():                   
  for sent in example_sentences:
    print(preprocess(sent))
    print(torch.exp(model(preprocess(sent))))

tensor([77, 46,  2, 33, 74])
tensor([9.9984e-01, 1.6109e-04])
tensor([77, 74,  2, 33, 46])
tensor([9.9984e-01, 1.6109e-04])
